# Flomo Intergration（闪念胶囊）

## Setup

In [1]:
import os
import tiktoken
import requests

from IPython.display import display
import ipywidgets as widgets

import speech_recognition as sr
import pyaudio
import wave

import promptlayer
promptlayer.api_key = os.getenv("PROMPTLAYER_API_KEY")

openai = promptlayer.openai
openai.api_key = os.getenv("OPENAI_API_KEY")

flomo_api = os.getenv("FLOMO_API")

In [2]:
# MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4"
TOKEN_LIMIT = 4096

## Utils

In [3]:
def num_tokens_from_string(string, model=MODEL):
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(string))

In [4]:
def num_tokens_from_messages(messages, model=MODEL):
    encoding = tiktoken.encoding_for_model(model)

    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    
    return num_tokens

In [5]:
def show_messages(messages):
    for message in messages:
        if 'name' not in message:
            print(f"{message['role']}:\n{message['content']}\n")
        else:
            print(f"{message['role']} \ {message['name']}:\n{message['content']}\n")

In [6]:
def reverse_roles(messages):
    reversed_messages = []
    for message in messages:
        if message["role"] == "assistant":
            message["role"] = "user"
        elif message["role"] == "user":
            message["role"] = "assistant"
        reversed_messages.append(message)
    return reversed_messages

In [7]:
def question_answering(prompt, system_message="You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.", temperature=0.0, pl_tags=["test"]):
    
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt},
    ]
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages,
        temperature=temperature,
        pl_tags=pl_tags,
    )
    
    return response["choices"][0]["message"]["content"]

In [8]:
def conversation(prompt, messages, temperature=0.0, pl_tags=["test"]):
    
    messages = messages + [{"role": "user", "content": prompt}]
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages,
        temperature=temperature,
        pl_tags=pl_tags,
    )

    messages.append({"role": "assistant", "content": response["choices"][0]["message"]["content"]})

    show_messages(messages)
    
    return messages

## Source

In [9]:
with open("data/selected_memos.txt", "r", encoding="utf-8") as f:
    memos = f.read()

with open("data/tags.txt", "r", encoding="utf-8") as f:
    tags = f.read()

In [10]:
instruction = f"""
我需要你作为一个memo生成器，帮我从比较随意的、口语化的、不严谨的语言中，去掉一些不必要的部分，稍微整理得结构化一些，然后按照我的口吻（下面会给出示例），生成一个memo。并打上1-3个tag。

我目前总共有这些tag（“/”代表子tag）：
{tags}

以下是我以前的一些memo，你可以参考一下：
{memos}
"""

def generate_memo(original_content, temperature=0.0, pl_tags=["flomo-intergration"]):
    messages = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": f"Original content:\n{original_content}\n\nMemo:"},
    ]

    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages,
        temperature=temperature,
        pl_tags=pl_tags,
    )

    return response["choices"][0]["message"]["content"]

In [11]:
def send_memos_to_flomo(memo):
    response = requests.post(flomo_api, {"content": memo})

    if response.status_code == 200:
        data = response.json()
        print('\n已发送到Flomo！')
    else:
        print('\n请求失败：', response.status_code)

In [12]:
original_content = '我刚刚在想，能不能做一个闪念胶囊的产品功能出来，应该会很好用。感觉不错的，主要就是结合语音功能来记录，然后可以随时随地听到，这样就不用担心忘记了。'
generated_memo = generate_memo(original_content)
print(generated_memo)
send_memos_to_flomo(generated_memo)

闪念胶囊产品功能想法
考虑开发一个闪念胶囊功能，通过结合语音记录，让用户随时随地记录和回听自己的想法，避免遗忘。
#想法 #生活/产品

已发送到Flomo！


In [13]:
def wav_to_text(wav_file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(wav_file_path) as audio_file:
        audio_data = recognizer.record(audio_file)
        try:
            text = recognizer.recognize_google(audio_data, language="zh-CN")
            print("\n识别成功！语音内容：", text)
            return text
        except sr.UnknownValueError:
            print("\n语音无法识别，请重试！")
        except sr.RequestError as e:
            print("\nCould not request results from Google Speech Recognition service; {0}".format(e))

def record_from_mic(button):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("开始录音，请说话...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("\n录音结束，识别中...")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open('data/audio.wav', 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    original_content = wav_to_text('data/audio.wav')
    generated_memo = generate_memo(original_content)
    print('\n生成Memo：\n' + generated_memo)
    send_memos_to_flomo(generated_memo)


record_button = widgets.Button(description="开始录音")
record_button.on_click(record_from_mic)
display(record_button)

Button(description='开始录音', style=ButtonStyle())

开始录音，请说话...

录音结束，识别中...

识别成功！语音内容： 你好这是我的第1条录音那么我也不知道要说什么

生成Memo：
第1条录音
今天进行了第一次录音尝试，内容较为随意，未确定具体话题。
#杂记

已发送到Flomo！
